<h2 align="center"> Data Mining and Machine Learning </h2>
<h3 align="center"> Final Project </h3>
<h2 align="center"> <b> <i> CrashSpot </i> </b> </h2>
<h4 align="center"> Lorenzo Ceccanti matr. 564490 </h4>

### <b> Data Integration </b>

In [42]:
editedDataset_folder = "../editedDataset"
dataset_folder = "../dataset"

In [43]:
import os
import pandas as pd

df = pd.read_csv(os.path.join(editedDataset_folder, 'CLEANED_brasilEnglishAggr.csv'))
pd.set_option('display.max_columns', None)
df.head(3)

,date,week_day,hour,state,road_id,km,city,cause_of_accident,type_of_accident,victims_condition,weather_timestamp,road_direction,weather_condition,road_type,people,deaths,slightly_injured,severely_injured,uninjured,unharmed,total_injured,vehicles_involved,latitude,longitude,regional,police_station
0,2017-01-01,sunday,01:45:00,RS,116,"34,9",VACARIA,Mechanical loss/defect of vehicle,Rear-end collision,With injured victims,Night,Decreasing,Clear sky,Simple,6,0,4,0,2,0,4,2,-28.50712,-50.94118,SPRF-RS,DEL05-RS
1,2017-01-01,sunday,01:00:00,PR,376,636,TIJUCAS DO SUL,Incompatible velocity,Run-off-road,With dead victims,Night,Increasing,Drizzle,Double,2,1,0,0,1,0,0,2,-25.75400,-49.12660,SPRF-PR,DEL01-PR
2,2017-01-01,sunday,04:40:00,BA,101,65,ENTRE RIOS,Driver was sleeping,Head-on collision,With dead victims,Sunrise,Decreasing,Cloudy,Simple,5,1,1,1,2,0,2,2,-11.96180,-38.09530,SPRF-BA,DEL01-BA


In [44]:
pd.reset_option('display.max_columns', None)

Let's inspect the dataset in which the granularity is per occupant, which will be useful for the second part of my project.

This dataset is not in English, so first it requires a little bit of translation

The source of data however it's the same. Each file contains different years.

<b> Problem </b>: If I try to import directly the dataset `BRASIL_RAW` we obtain an UnicodeDecodeError. We discover that the encoding for the dataset is not UTF-8.

In [45]:
import os
import chardet

for i in range(2017,2024):
    with open(os.path.join(f'{dataset_folder}/BRASIL_RAW/acidentes', f'acidentes{i}.csv'), 'rb') as f:
        result = chardet.detect(f.read(10000))  # leggi i primi 10k byte
        print(f'Encoding for acidentes{i}: ' + result['encoding'])

Encoding for acidentes2017: ISO-8859-1
Encoding for acidentes2018: ISO-8859-1
Encoding for acidentes2019: ISO-8859-1
Encoding for acidentes2020: ISO-8859-1
Encoding for acidentes2021: ISO-8859-1
Encoding for acidentes2022: ISO-8859-1
Encoding for acidentes2023: ISO-8859-1


In [46]:
# This little script converts the original brasil_raw into UTF-8 encoding

# Checking if the new directory we want to create already exists
out_dir = f"{editedDataset_folder}/UTF_acidentes"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
    for i in range(2017,2024):
        src = os.path.join(f'{dataset_folder}/BRASIL_RAW/acidentes', f'acidentes{i}.csv')
        dst = os.path.join(f'{editedDataset_folder}/UTF_acidentes', f'utf_acidentes{i}.csv')
        # The \ operator is useful to truncate the writing of the code in multiple line for
        # improving the readability of the code
        with open(src, "r", encoding="iso-8859-1", errors="strict") as fin, \
            open(dst, "w", encoding="utf-8", newline="") as fout:
            for line in fin:
                fout.write(line)

In [47]:
arr_df_full = []
files_to_inspect = ['2017', '2018', '2019', '2020', '2021', '2022', '2023']

for y in files_to_inspect:
    # Step 1: Importing the dataset of year y
    df_full = pd.read_csv(os.path.join(f'{editedDataset_folder}/UTF_acidentes', f'utf_acidentes{y}.csv'), sep=";", dtype={22: "string", 23:"string", 25:"string"})

    # Step 2: Translation of the category names in English
    # Taking the first attributes until road_delineation

    # Translation of the category names in English
    # Taking the first attributes until road_delineation

    en_attrNames_head = (df.loc[:,:'people'].columns).tolist()
    # We need the previous labels
    df_full_columns = df_full.columns.tolist()

    # With this selection we're sure to substitute only the names in the first part
    df_full_columns[2:17] = en_attrNames_head
    # Writing the translation for the central part of the columns
    df_full_columns[17:26] = ['without_passengers', 'veichle_id', 'veichle_type', 'veichle_brand', 'veichle_manufacturing_year', 'person_kind', 'person_condition', 'person_age', 'person_sex']
    df_full_columns[26:30] = ['person_is_unharmed', 'person_is_slightly_injured', 'person_is_severely_injured', 'person_is_dead']
    df_full_columns[32] = 'police_station'
    # Applying the translation to the DataFrame
    df_full.columns = df_full_columns

    # Step 3: Translating the instances values in English (attribute per attribute)

    without_passengers_map = {
        'Não': 'No',
        'Sim': 'Yes'
    }
    df_full["without_passengers"] = df_full["without_passengers"].replace(without_passengers_map)

    vehicle_type_map = {
        "Automóvel": "Car",
        "Motocicleta": "Motorcycle",
        "Semireboque": "Semi-trailer",
        "Caminhonete": "Pickup truck",
        "Caminhão-trator": "Tractor-trailer truck",
        "Caminhão": "Truck",
        "Ônibus": "Bus",
        "Camioneta": "Van",
        "Motoneta": "Scooter",
        "Utilitário": "Utility vehicle",
        "Bicicleta": "Bicycle",
        "Micro-ônibus": "Minibus",
        "Reboque": "Trailer",
        "Outros": "Others",
        "Ciclomotor": "Moped",
        "Carroça-charrete": "Cart-wagon",
        "Trator de rodas": "Wheeled tractor",
        "Motor-casa": "Motorhome",
        "Triciclo": "Tricycle",
        "Trem-bonde": "Tram",
        "Trator de esteira": "Crawler tractor",
        "Trator misto": "Backhoe loader",
        "Carro de mão": "Wheelbarrow",
        "Chassi-plataforma": "Chassis platform",
        "Quadriciclo": "Quadricycle"
    }
    df_full["veichle_type"] = df_full["veichle_type"].replace(vehicle_type_map)

    df_full['veichle_brand'] = df_full["veichle_brand"].replace({
        "Não Informado/Não Informado": pd.NA,
        "NA/NA": pd.NA
    })

    df_full['veichle_manufacturing_year'] = df_full["veichle_manufacturing_year"].replace(0,pd.NA)

    person_kind_map = {
        'Condutor': 'Driver',
        'Passageiro': 'Passenger',
        'Pedestre': 'Pedestrian',
        'Testemunha': 'Withness',
        'Cavaleiro': 'Knight'
    }
    df_full["person_kind"] = df_full["person_kind"].replace(person_kind_map)

    person_sex_map = {
        'Masculino': 'M',
        'Feminino': 'F',
        'Não Informado': pd.NA,
        'Ignorado': pd.NA
    }
    df_full["person_sex"] = df_full["person_sex"].replace(person_sex_map)

    person_condition_map = {
        'Ileso': 'Unharmed',
        'Lesões Leves': 'Slightly Injured',
        'Lesões Graves': 'Severely Injured',
        'Não Informado': pd.NA,
        'Óbito': 'Dead'
    }
    df_full["person_condition"] = df_full["person_condition"].replace(person_condition_map)

    # Step 4: Converting latitude, longitude from object to float64
    # We need also to round to 5 digits in order to make the join operation successful
    attr_to_conv = ["latitude", "longitude"]
    for attr in attr_to_conv:
        df_full[attr] = df_full[attr].astype(str).str.replace(",", ".").astype(float).round(5)
        df[attr] = df[attr].round(5)

    # Step 5: We sort by inverse_data, hour, city (in place)
    df.sort_values(by=['date', 'hour', 'city'], inplace=True)
    df_full.sort_values(by=['date', 'hour', 'city'], inplace=True)

    # Step 6: From df_full we remove all the duplicate attributes that coincides in both the level of granularity
    indexes = {0,2} # we drop all the head attributes exception made the ones useful for the join
    en_attrNames_head = [x for i,x in enumerate(en_attrNames_head) if i not in indexes]
    df_full = df_full.drop(columns=en_attrNames_head)

    # Step 7: We join the two tables with different granularity
    df_joined = pd.merge(df, df_full, on=['latitude', 'longitude', 'date', 'hour'])
    arr_df_full.append(df_joined)

`arr_df_full` is an array of DataFrames. In each DataFrame one instance is a person or a withness involved in an accidents, but also has the details about the accident.

Now, let's produce the file `INTEGRATION_brasilEnglishFull.csv`, which is the result of the join operation we've performed so far.

In [48]:
import os
out_dir = editedDataset_folder
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

file_path = os.path.join(out_dir, 'INTEGRATION_brasilEnglishFull.csv')
if not os.path.exists(file_path):
    df_all = pd.concat(arr_df_full, ignore_index=True)
    df_all.to_csv(file_path, index=False, encoding='utf-8')

### Road feature integration

Now, we'll exploit Overpass API to gather some additional details about the road.

In [49]:
# Here we are specifying the range for the indexes of the DataFrame in which
# we are interested on to look for the speedlimits.
start_indexes = [400,100000]
end_indexes = [500,100100]

integrate_road = False

In [50]:
# Attention: takes a lot of time!

import numpy as np
import requests
from json import JSONDecodeError
from time import sleep

url = "http://overpass-api.de/api/interpreter"
overpass_dfs = []

if integrate_road:
    
    for start_index, end_index in zip(start_indexes, end_indexes):
        df_coords_unique = df.iloc[start_index:end_index+1].loc[:,'latitude':'longitude'].drop_duplicates()
        df_coords_unique

        tags = []
        for index, row in df_coords_unique.iterrows():
            latitude = row['latitude']
            longitude = row['longitude']

            query = f"""
            [out:json][timeout:25];
            way(around:8,{latitude},{longitude})[highway];
            out tags;
            """
            
            # retry loop
            max_retries = 3
            for attempt in range(max_retries):
                try:
                    response = requests.get(url, params={'data': query}, timeout=40)
                    if response.status_code == 200:
                        break
                    else:
                        print(f"Attempt {attempt+1}/{max_retries} failed with status {response.status_code}")
                except requests.exceptions.RequestException as e:
                    print(f"Tentativo {attempt+1}/{max_retries} fallito con errore: {e}")
                sleep(2)
            else:
                # se esco dal ciclo senza break, vuol dire che tutti i tentativi sono falliti
                print(f"All attempts failed at {index}, skipping...")
                continue
            response = requests.get(url, params={'data': query})
            
            print(f"Processing index: {index}")
            try:
                data = response.json()
                if data["elements"]:
                    elements_arr = data["elements"]
                    for elem in elements_arr:
                        if elem["tags"]:
                            dict = elem["tags"]
                            # I want to keep only: maxspeed, lanes, name, operator, toll, surface, oneway
                            # I use a list comprehesion
                            keep = {"maxspeed", "lanes", "name", "operator", "toll", "surface", "oneway"}
                            dict = {k: v for k, v in elem["tags"].items() if k in keep}
                            dict['latitude'] = latitude
                            dict['longitude'] = longitude
                            tags.append(dict)
            except JSONDecodeError:
                print("Returned JSON Decode Error.. Continuing..")
                continue

        overpass_df = pd.DataFrame(tags)
        overpass_dfs.append(overpass_df)

Output when `integrate_road = True`:

```text
Processing index: 24936
Attempt 1/3 failed with status 429
Processing index: 24916
Processing index: 50
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 504
Processing index: 56
Returned JSON Decode Error.. Continuing..
Processing index: 24918
Processing index: 24939
Processing index: 25371
Returned JSON Decode Error.. Continuing..
Processing index: 49
Processing index: 3370
Processing index: 43
Attempt 1/3 failed with status 429
Processing index: 24906
Processing index: 24927
Attempt 1/3 failed with status 429
Processing index: 40777
Processing index: 72800
Processing index: 24919
Processing index: 45
Processing index: 48
Processing index: 24910
Processing index: 46
Returned JSON Decode Error.. Continuing..
Processing index: 24915
Returned JSON Decode Error.. Continuing..
Processing index: 24917
Processing index: 48883
Returned JSON Decode Error.. Continuing..
Processing index: 57
Processing index: 24930
Processing index: 47
Returned JSON Decode Error.. Continuing..
Processing index: 24946
Processing index: 24929
Attempt 1/3 failed with status 429
Processing index: 24928
Processing index: 24965
Returned JSON Decode Error.. Continuing..
Processing index: 24954
Processing index: 24925
Processing index: 54
Processing index: 89360
Processing index: 24922
Returned JSON Decode Error.. Continuing..
Processing index: 24920
Processing index: 24935
Processing index: 24923
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 504
Processing index: 24933
Processing index: 53
Returned JSON Decode Error.. Continuing..
Processing index: 24926
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 504
Attempt 2/3 failed with status 504
Processing index: 24937
Processing index: 51
Attempt 1/3 failed with status 429
Processing index: 24931
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 504
Processing index: 24969
Processing index: 24924
Returned JSON Decode Error.. Continuing..
Processing index: 31290
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 429
Attempt 2/3 failed with status 504
Attempt 3/3 failed with status 504
All attempts failed at 25023, skipping...
Processing index: 24932
Returned JSON Decode Error.. Continuing..
Processing index: 24941
Processing index: 24934
Attempt 1/3 failed with status 429
Attempt 2/3 failed with status 504
Processing index: 52
Processing index: 24940
Returned JSON Decode Error.. Continuing..
Processing index: 24938
Returned JSON Decode Error.. Continuing..
Processing index: 25263
Attempt 1/3 failed with status 429
Processing index: 55
Processing index: 24948
Processing index: 24962
Processing index: 25000
Processing index: 24527
Processing index: 24993
Processing index: 24942
Attempt 1/3 failed with status 504
Processing index: 24944
Processing index: 24947
Attempt 1/3 failed with status 504
Attempt 2/3 failed with status 504
Processing index: 70
Processing index: 24945
Returned JSON Decode Error.. Continuing..
Processing index: 24998
Returned JSON Decode Error.. Continuing..
Processing index: 24960
Attempt 1/3 failed with status 429
Processing index: 65
Attempt 1/3 failed with status 429
Processing index: 24992
Processing index: 24955
Processing index: 58
Processing index: 24967
Returned JSON Decode Error.. Continuing..
Processing index: 24964
Processing index: 24968
Processing index: 25009
Processing index: 24975
Processing index: 24949
Returned JSON Decode Error.. Continuing..
Processing index: 24951
Attempt 1/3 failed with status 429
Processing index: 24958
Attempt 1/3 failed with status 504
Processing index: 24970
Processing index: 60
Attempt 1/3 failed with status 429
Attempt 2/3 failed with status 504
Processing index: 24961
Processing index: 24952
Attempt 1/3 failed with status 429
Processing index: 24956
Processing index: 24977
Attempt 1/3 failed with status 504
Processing index: 24999
Returned JSON Decode Error.. Continuing..
Processing index: 24985
Attempt 1/3 failed with status 504
Processing index: 24953
Processing index: 24957
Processing index: 24959
Processing index: 24972
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 429
Processing index: 61
Processing index: 24966
Processing index: 24986
Processing index: 35394
Processing index: 24982
Returned JSON Decode Error.. Continuing..
Processing index: 24963
Processing index: 24974
Attempt 1/3 failed with status 504
Processing index: 66656
Processing index: 72
Returned JSON Decode Error.. Continuing..
Processing index: 92195
Processing index: 92196
Returned JSON Decode Error.. Continuing..
Processing index: 93668
Processing index: 92197
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 504
Processing index: 115032
Processing index: 115038
Attempt 1/3 failed with status 504
Processing index: 115034
Processing index: 92198
Attempt 1/3 failed with status 504
Processing index: 115033
Processing index: 115051
Returned JSON Decode Error.. Continuing..
Processing index: 115041
Processing index: 115060
Attempt 1/3 failed with status 504
Attempt 2/3 failed with status 504
Processing index: 115043
Processing index: 92200
Attempt 1/3 failed with status 429
Processing index: 92199
Processing index: 115047
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 504
Attempt 2/3 failed with status 504
Processing index: 92201
Processing index: 92224
Attempt 1/3 failed with status 429
Attempt 2/3 failed with status 429
Processing index: 115039
Processing index: 115045
Attempt 1/3 failed with status 429
Processing index: 115066
Processing index: 115063
Processing index: 115046
Processing index: 115056
Returned JSON Decode Error.. Continuing..
Processing index: 115049
Processing index: 115042
Attempt 1/3 failed with status 429
Processing index: 115040
Returned JSON Decode Error.. Continuing..
Processing index: 115044
Processing index: 126237
Processing index: 115048
Returned JSON Decode Error.. Continuing..
Processing index: 115092
Processing index: 92205
Processing index: 115062
Processing index: 115050
Attempt 1/3 failed with status 504
Processing index: 115052
Attempt 1/3 failed with status 429
Processing index: 115053
Processing index: 115054
Returned JSON Decode Error.. Continuing..
Processing index: 115057
Processing index: 92219
Processing index: 115055
Processing index: 92202
Returned JSON Decode Error.. Continuing..
Processing index: 115058
Processing index: 115073
Returned JSON Decode Error.. Continuing..
Processing index: 115059
Attempt 1/3 failed with status 504
Processing index: 115061
Returned JSON Decode Error.. Continuing..
Processing index: 115067
Processing index: 115064
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 504
Attempt 2/3 failed with status 504
Processing index: 115065
Processing index: 115104
Returned JSON Decode Error.. Continuing..
Processing index: 92206
Processing index: 92208
Attempt 1/3 failed with status 429
Processing index: 92228
Attempt 1/3 failed with status 504
Processing index: 92225
Returned JSON Decode Error.. Continuing..
Processing index: 115070
Processing index: 115076
Attempt 1/3 failed with status 429
Attempt 2/3 failed with status 504
Processing index: 115068
Processing index: 92209
Attempt 1/3 failed with status 429
Attempt 2/3 failed with status 504
Processing index: 115083
Processing index: 115077
Attempt 1/3 failed with status 429
Attempt 2/3 failed with status 429
Processing index: 115069
Processing index: 115071
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 429
Processing index: 92212
Returned JSON Decode Error.. Continuing..
Processing index: 115120
Returned JSON Decode Error.. Continuing..
Processing index: 115072
Processing index: 115075
Returned JSON Decode Error.. Continuing..
Processing index: 115080
Attempt 1/3 failed with status 504
Processing index: 115096
Returned JSON Decode Error.. Continuing..
Processing index: 92213
Returned JSON Decode Error.. Continuing..
Processing index: 92223
Processing index: 115110
Returned JSON Decode Error.. Continuing..
Processing index: 115078
Returned JSON Decode Error.. Continuing..
Processing index: 136457
Processing index: 115106
Processing index: 115082
Attempt 1/3 failed with status 504
Processing index: 115079
Processing index: 118541
Processing index: 115101
Returned JSON Decode Error.. Continuing..
Processing index: 115081
Processing index: 115087
Returned JSON Decode Error.. Continuing..
Processing index: 115090
Processing index: 115102
Attempt 1/3 failed with status 429
Processing index: 115084
Processing index: 115085
Processing index: 115091
Processing index: 115094
Returned JSON Decode Error.. Continuing..
Attempt 1/3 failed with status 429
Processing index: 115107
Processing index: 115089
Returned JSON Decode Error.. Continuing..
Processing index: 115086
Processing index: 107543
Processing index: 115088
Returned JSON Decode Error.. Continuing..
Processing index: 115095
Processing index: 107547
Attempt 1/3 failed with status 504
Processing index: 115093
Processing index: 115100
Attempt 1/3 failed with status 504
Processing index: 92220
Returned JSON Decode Error.. Continuing..
Processing index: 92217
Processing index: 92214
Attempt 1/3 failed with status 429
Processing index: 119972
Processing index: 115112
Returned JSON Decode Error.. Continuing..
Processing index: 115108
Returned JSON Decode Error.. Continuing..
```

In [51]:
if overpass_dfs:
    overpass_dfs[0].shape[0]

Output when `integrate_road = True`:

```text
74
```

In [52]:
if overpass_dfs:
    overpass_dfs[1].shape[0]

Output when `integrate_road = True`:

```text
60
```

From the 100 instances we start from, only 60-70 satisfy the OverPass API query

In [53]:
overpass_dfs_clean = []
for overpass_df in overpass_dfs:
    overpass_df_clean = overpass_df.dropna(subset=['lanes', 'oneway', ])
    overpass_dfs_clean.append(overpass_df_clean)

In [54]:
if overpass_dfs_clean:
    overpass_dfs_clean[0].shape[0]

Output when `integrate_road = True`:
```text
34
```

In [55]:
if overpass_dfs_clean:
    overpass_dfs_clean[1].shape[0]

Output when `integrate_road = True`:
```text
35
```

Since in both cases we obtained that from 100 instances we begin from, only 25-30 of them have associated information concerning the speedlimit and other road features I decide to not consider this aspect in the project. 